In [340]:
import pandas as pd

In [341]:
# REad property files to get the csv file paths
d = {}
with open('properties.txt') as f:
    for line in f:
        key, value = line.split('=')
        d[key.strip()] = value.strip()


In [342]:
csv_path= d['csv_path']



In [343]:
loan= d['loan']
loan_csv = csv_path + loan
# Import the data into a Pandas DataFrame
loan_df = pd.read_csv(loan_csv)


In [344]:
covanent =d['covanent']
covanent_csv= csv_path + covanent 

# Import the data into a Pandas DataFrame
covanent_df = pd.read_csv(covanent_csv)

covanent_df.fillna(0)

,facility_id,max_default_likelihood,bank_id,banned_state
0,1,0.09,3,ND
1,5,0.09,1,LA
2,4,0.09,4,PA
3,7,0.09,5,RI
4,3,0.00,4,ND
5,8,0.09,1,OK
6,11,0.00,3,KS
7,11,0.09,3,TN
8,1,0.00,3,MO
9,3,0.00,4,NE


In [345]:
facility= d['facility']
facility_csv =csv_path + facility

# Import the data into a Pandas DataFrame
facility_df = pd.read_csv(facility_csv)


In [346]:
facility_df=facility_df.rename(columns = {"id":"facility_id"})



In [347]:
#Merge the 2 DF and remove duplicate columns
result = pd.merge(facility_df, covanent_df, on=["facility_id"])
del result['bank_id_x']
result=result.rename(columns = {"bank_id_y":"bank_id"})
result_df =pd.DataFrame(result)
result_df.fillna(0 ,inplace=True)



In [348]:

filtered_cov_df= pd.DataFrame(columns=['loan_id', 'facility_id','loan_amt', 'amount', 'interest_rate'])



In [350]:
#Calculate assignments
for loan_id, loan_amt, default_rate, state in zip(loan_df['id'],loan_df['amount'], loan_df['default_likelihood'], loan_df['state']):
    #Apply covanents and remove all the loan amount that a ficilty cant fund
    cov_subset_df = pd.DataFrame(result_df.loc[(result_df['max_default_likelihood'] >= default_rate) &(result_df['banned_state'] !=state ) & (result_df['amount'] >= loan_amt) ]  ) 
    if( cov_subset_df.shape[0] > 1):
        minimum = cov_subset_df['interest_rate'].min()
        #If multiple facilities are applied to a loan use the facilit with cheapest rates to assign to a loan
        df_not_minimum = cov_subset_df[ cov_subset_df['interest_rate'] > minimum ] 
        cov_subset_df = cov_subset_df.drop(df_not_minimum.index, axis=0)  
    
    filtered_cov_df = filtered_cov_df.append({'loan_id' :loan_id, 'facility_id': cov_subset_df['facility_id'].values[0], 'loan_amt':loan_amt, 'amount': cov_subset_df['amount'].values[0], 'interest_rate': cov_subset_df['interest_rate'].values[0]}, ignore_index=True)

agreement_df = pd.DataFrame(filtered_cov_df)
del agreement_df['loan_amt']
del agreement_df['amount']
del agreement_df['interest_rate']
assignments= d['assignments']
assignments_csv =csv_path + assignments
agreement_df.to_csv(assignments_csv, index=False, header=True)

    
   

In [352]:
count = pd.DataFrame(filtered_cov_df.facility_id.value_counts())


In [353]:
#Calculate yeild
final_yeild_df=pd.DataFrame(columns=['facility_id', 'expected_yield'])
for index, row in count.iterrows() :
    yield_df = pd.DataFrame(filtered_cov_df.loc[(filtered_cov_df['facility_id'] == index)])
   
    fac_rate = facility_df[facility_df['facility_id'] ==index]['interest_rate'].values[0]   
    expected_yeild= 0
    for loan_id in zip( yield_df['loan_id']) :  
       
        default_liklyhood = loan_df[loan_df['id']==loan_id[0]]['default_likelihood'].values[0]
        
        loan_int_rate= loan_df[loan_df['id'] ==loan_id[0]]['interest_rate'].values[0]
       
        loan_amt= loan_df[loan_df['id'] ==loan_id[0]]['amount'].values[0]  
        
        yeild = ( 1-default_liklyhood)*loan_int_rate*loan_amt-default_liklyhood*loan_amt-fac_rate*loan_amt
       
        expected_yeild = expected_yeild +  yeild
    final_yeild_df= final_yeild_df.append( {'facility_id': index , 'expected_yield': expected_yeild}, ignore_index=True)

yeilds= d['yeilds']
yeild_csv =csv_path + yeilds
agreement_df.to_csv(yeild_csv, index=False, header=True)


,facility_id,expected_yield
0,1.0,7779638.808
1,4.0,113060.357
